In [1]:
import uuid
import time
import pickle
import sys
import gym.spaces
import itertools
import numpy as np
import random
import tensorflow                as tf
import tensorflow.contrib.layers as layers
from collections import namedtuple
import TicTacToe
from collections import Counter
import Players



/Users/christophermiller/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [31]:
tf.reset_default_graph()

def TicTacToe_model(placeholder, num_actions, scope, reuse=tf.AUTO_REUSE):
    # A model for a TicTacToe q-function
    placeholder = tf.contrib.layers.flatten(placeholder)
    with tf.variable_scope(scope, reuse=reuse):
        out = placeholder
        out = tf.cast(out, tf.float32)
        out = tf.layers.dense(out, 64  , bias_initializer = tf.zeros_initializer(), activation = tf.nn.softmax)
        out = tf.layers.dense(out, 64  , bias_initializer = tf.zeros_initializer(), activation = tf.nn.softmax)
        out = tf.layers.dense(out, 9 , kernel_initializer = tf.zeros_initializer(), bias_initializer = tf.zeros_initializer(), activation = None)
    return out

    
# def update_old_model_initialize_new_model(old_model, new_model, board_placeholder, sess, update_iter):
#     old_model = new_model
#     old_model_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="model" +str(update_iter-1))
#     
#     new_model = TicTacToe_model(board_placeholder, 9, scope = "model" + str(update_iter))
#     
#     new_model_vars =  tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="model" +str(update_iter))
#     sess.run(tf.variables_initializer(new_model_vars))
#     
#     for var_old, var_new in zip(old_model_vars, new_model_vars):
#         var_new = sess.run(var_old)
# #        print(sess.run(var_old))
# #        print(sess.run(var_new))
# #    print(sess.run(old_model, feed_dict = {board_placeholder: [[[1,0,0],[0,0,0],[0,0,0]]]})[0,:3])
# #    print(sess.run(new_model, feed_dict = {board_placeholder: [[[1,0,0],[0,0,0],[0,0,0]]]})[0,:3])
#     return old_model, new_model
# 
# def update_and_duplicate_old_model():
#     old_model = new_model
#     
#     variables = tf.trainable_variables()
#     for var1 in variables:
#         if "pred/" in var1.name:
#             trained_var = [var2 for var2 in tf.global_variables() if var2.op.name in str.replace(var1.name, "pred/", "")][0]
#             value = sess.run(trained_var)
#             sess.run(tf.assign(var1, value))


def sample_action(model, mask_placeholder):
    out = model
    dist = tf.distributions.Categorical(probs=maskedSoftmax(out, mask_placeholder))
    return dist.sample()
    
    
"""Code from online"""
def maskedSoftmax(logits, mask):
    """
    Masked softmax over dim 1
    :param logits: (N, L)
    :param mask: (N, L)
    :return: probabilities (N, L)
    """
    indices = tf.where(mask)
    values = tf.gather_nd(logits, indices)
    denseShape = tf.cast(tf.shape(logits), tf.int64)
    
    """THIS IS THE KEY: tensorflow will automatically set output probabilities to zero of undesignated entries in sparse vector"""
    sparseResult = tf.sparse_softmax(tf.SparseTensor(indices, values, denseShape))
    
    result = tf.scatter_nd(sparseResult.indices, sparseResult.values, sparseResult.dense_shape)
    result.set_shape(logits.shape)
    return result

# def sample_trajectory_random_opponent(new_model, sess, env):
#     new_sample = sample_action(new_model)
#     obs, acs, rewards = [], [], []
#     seed = np.random.randint(0,2)
#     ob = env.reset()
#     if seed % 2 == 0:
#         while True:
#             obs.append(ob)
#             action = sess.run(new_sample, feed_dict={board_placeholder: [ob]})
#             acs.append(action)
#             ob, rew, done, _ = env.step(action[0])
#             rewards.append(rew)
#             
#             if done:
#                 break
#             
#             action = [env.action_space.sample()]
#             ob, rew, done, _ = env.step(action[0])
#     else:
#         while True:       
#             action = [env.action_space.sample()]
#             ob, rew, done, _ = env.step(action[0])
#    
#             obs.append(ob)
#             action = sess.run(new_sample, feed_dict={board_placeholder: [ob]})
#             acs.append(action)
#             ob, rew, done, _ = env.step(action[0])
#             rewards.append(rew)
#             
#             if done:
#                 break
#                 
#     path = {"observation" : np.array(obs, dtype=np.float32), 
#                 "reward" : np.array(rewards, dtype=np.float32), 
#                 "action" : np.array(acs, dtype=np.float32)}
# #    print(str(len(path["observation"])))
#     return path
    
def batch_rollout(player,opponent, env, max_time_steps = 100, exploration_on =False, epsilon =.1):
    paths = []
    batch_winners = Counter({0: 0, 1: 0, 2:0})
    time_steps = 0
    while time_steps < max_time_steps:
        path = sample_trajectory(player,opponent,env, exploration_on, epsilon)
        paths += [path]
        batch_winners[env.current_winner] +=1
        time_steps += len(path['observation'])
    return paths, batch_winners
    
    
    
def sample_trajectory(player, opponent, env, exploration_on = False, epsilon = .1):
    """player:   realization of Player.Player abstract class
       opponent: realization of Player.Player abstract class
       env:      environment which follows open ai gym environment structure and has a current_player int either 1 or 2
    
       realizes a rollout of env using player and opponent policy
       
       ouputs a path dictionary with keys: observation, reward, action
       Each yields a 1D np array which shows the observation, reward, action pair at every point of the rollout
    """
    
    obs, acs, rewards, masks = [], [], [], []
    ob = env.reset()
    done = False
    player_has_acted = False
    action = None
    
    #Do rest of moves
    while not done:
        #Get current observation of current player
        ob = env.get_observation(env.current_player)
        legal_moves = env.legal_moves()
        if env.current_player == 1:
            #Reward is recorded as results of state,action pair... need to check player 1 has acted already
            if player_has_acted:
                rewards.append(env.get_reward(1))
            else:
                player_has_acted = True
                
            action = player.policy(np.array([ob]), np.array([legal_moves]))
            if exploration_on:
                legal_options = np.array(legal_moves)
                action = [random.choice(np.nonzero(legal_options)[0])]
            obs.append(ob)
            acs.append(action[0])
            masks.append(legal_moves)
        else:
            action = opponent.policy(np.array([ob]), np.array([legal_moves]))
        done, _ = env.step(action[0]) 

    #Need to record final reward for player 1
    rewards.append(env.get_reward(1))
    
    path = {"observation" : np.array(obs, dtype=np.int32), 
                "reward" : np.array(rewards, dtype=np.float32), 
                "action" : np.array(acs, dtype=np.int32),
                "mask" : np.array(masks, dtype=np.int32)}
    return path

    
    
    

# def sample_trajectories(player, opponent, env, min_timesteps_per_batch):
#     paths =[]
#     timesteps_this_batch = 0
#     while True:
#         path = sample_trajectory(player, opponent, env)
#         paths.append(path)
#         timesteps_this_batch += len(path['observation'])
#         if timesteps_this_batch > min_timesteps_per_batch:
#             break
#     return paths
    
def sum_of_rewards(paths, gamma = .6): 
    re_n = [path["reward"] for path in paths]
    q_n = []
    for seq_of_rewards in re_n:
        for t in range(len(seq_of_rewards)):
            weighted_sequence = seq_of_rewards[t:] * np.array([gamma**i for i in range(len(seq_of_rewards[t:]))])
            q_n.append(np.sum(weighted_sequence))
    adv_n = q_n
    return adv_n
        
def standardize_advantage(adv_n):
    adv_n = (adv_n - np.mean(adv_n)) 
    adv_n = adv_n * (1.0/(np.std(adv_n)+.0000001))
    return adv_n

def get_log_prob(model, action_placeholder, mask_placeholder):
    action_dim = 9 
    logits = model
    
    indices = tf.where(mask_placeholder)
    values = tf.gather_nd(logits, indices)
    denseShape = tf.cast(tf.shape(logits), tf.int64)
    
    """THIS IS THE KEY: tensorflow will automatically set output probabilities to zero of undesignated entries in sparse vector"""
    sparseResult = tf.sparse_softmax(tf.SparseTensor(indices, values, denseShape))
    
    probability_dist = tf.scatter_nd(sparseResult.indices, sparseResult.values, sparseResult.dense_shape)
#     probability_dist = probability_dist.set_shape(logits.shape)
    log_probability_dist = tf.scatter_nd(sparseResult.indices, tf.log(sparseResult.values), sparseResult.dense_shape)

    """Want to emulate this:"""
#     probability_dist = tf.nn.softmax(logits)
#     legal_pseudo_probability_dist = probability_dist*values
#     legalprobability_dist = tf.divide(legal_pseudo_probability_dist, tf.reduce_sum(legal_pseudo_probability_dist, axis= 1))
    
    prod = tf.multiply(probability_dist, tf.one_hot(action_placeholder, action_dim ))
    
    entropy = - tf.reduce_sum(probability_dist * log_probability_dist, axis = 1)
    
    
    
    log_prob = tf.log(tf.reduce_sum(prod , axis = 1 ))
#    log_prob = -tf.nn.sparse_softmax_cross_entropy_with_logits(labels= action_placeholder, logits= tf.SparseTensor(indices, values, denseShape))
    return log_prob, entropy

def loss_and_update_op(log_prob, entropy, adv_n, entropy_coeff = .1):
    loss = -tf.reduce_mean(log_prob * adv_n) -  entropy_coeff * entropy
    optimizer = tf.train.AdamOptimizer(5e-3)
    update_op = optimizer.minimize(loss)
    return loss, update_op, optimizer
    

In [5]:
tf.reset_default_graph()

#define the board, models *symbolically*
observation_placeholder = tf.placeholder(shape = [None, 3,3], dtype = tf.int32)
adv_n_placeholder = tf.placeholder(shape = [None], dtype = tf.float32)
action_placeholder = tf.placeholder(shape = [None], dtype = tf.int32)
mask_placeholder = tf.placeholder(shape=[None, 9], dtype = tf.int32)


model = TicTacToe_model(observation_placeholder, 9, scope = "model-1", reuse=tf.AUTO_REUSE)
#old_model = TicTacToe_model(board_placeholder, 9, scope = "model-2", reuse=tf.AUTO_REUSE)
model_input_s = sample_action(model, mask_placeholder)

#Define Loss functions *symbolically*
log_prob, entropy = get_log_prob(model, action_placeholder, mask_placeholder)
loss, update_op, optimizer = loss_and_update_op(log_prob, entropy, adv_n_placeholder, entropy_coeff = 0)

#start a session
sess =tf.Session()
sess.run(tf.global_variables_initializer())
#Defines player, opponent
player = Players.NN_Player(model, model_input_s, sess, observation_placeholder, mask_placeholder, duplicate=False)
opponent = Players.Random_Player()

#Loads old player,opponent
# temp_file_name = './bot_10_28_v6.ckpt'

#Want to duplicate session
# saver = tf.train.Saver()
# saver.restore(sess, temp_file_name)


# opponent = Players.NN_Player(model, model_input_s, sess, observation_placeholder, mask_placeholder)




#start an environment
env = TicTacToe.TicTacToe()

number_updates_per_expert_update = 5
number_expert_updates = 1000

for k in range(number_expert_updates):
    print("iteration number", k)
    
    batch_adv_n = []
    iteration_winners = Counter({0:0,1:0,2:0})
    
    tic = time.time()
    for i in range(number_updates_per_expert_update):
        paths, batch_winners = batch_rollout(player, opponent, env, max_time_steps=1000)
        iteration_winners += batch_winners
        
        adv_n = sum_of_rewards(paths)
        batch_adv_n = batch_adv_n + adv_n
        

        boards = np.concatenate([path['observation'] for path in paths])
        masks = np.concatenate([path['mask'] for path in paths])
        actions = np.squeeze(np.concatenate([path["action"] for path in paths])).astype(int)
        
        sess.run(update_op, feed_dict = {mask_placeholder: masks, adv_n_placeholder: adv_n, observation_placeholder: boards , action_placeholder: actions})
    
    
    #Unwind win data:
#     print(iteration_winners)
    print("mean adv", np.mean(batch_adv_n))
    print("iteration time", time.time() - tic)
#     print(paths[0])
    
    
    expert_player = Players.Expert_Player()
    _, expert_batch_winners = batch_rollout(player, expert_player, env, max_time_steps=900, exploration_on =True, )
    player_loss_percentage_vs_expert = expert_batch_winners[2]*1.0/(expert_batch_winners[0] + expert_batch_winners[1] + expert_batch_winners[2])
    print("loss percent vs expert", player_loss_percentage_vs_expert)
    opponent = Players.NN_Player(model, model_input_s, sess, observation_placeholder,mask_placeholder)
            
    


iteration number 0


KeyboardInterrupt: 

In [4]:
temp_file_name = './bot_10_28_v7.ckpt'

#Want to duplicate session
saver = tf.train.Saver()
saver.save(sess, temp_file_name)

'./bot_10_28_v6.ckpt'

In [9]:
temp_file_name = './bot_10_28_v3.ckpt'

#Want to duplicate session
saver = tf.train.Saver()
saver.restore(sess, temp_file_name)

INFO:tensorflow:Restoring parameters from ./bot_10_28_v3.ckpt


In [ ]:
human = Players.Human_Player()
sample_trajectory(human,opponent,env)


1
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Say something: 2
[2]
1
[[0. 0. 1.]
 [0. 2. 0.]
 [0. 0. 0.]]
Say something: 7
[7]
1
[[2. 0. 1.]
 [0. 2. 0.]
 [0. 1. 0.]]
Say something: 8
[8]
1
[[2. 2. 1.]
 [0. 2. 0.]
 [0. 1. 1.]]
Say something: 3
[3]
1
[[2. 2. 1.]
 [1. 2. 2.]
 [0. 1. 1.]]


In [19]:
tf.reset_default_graph()

#define the board, models
board_placeholder = tf.placeholder(shape = [None, 3,3], dtype = tf.int32)
adv_n_placeholder = tf.placeholder(shape = [None], dtype = tf.float32)
action_placeholder = tf.placeholder(shape = [None], dtype = tf.int32)
new_model = TicTacToe_model(board_placeholder, 9, scope = "model-1", reuse=tf.AUTO_REUSE)

#start a session
sess = tf.Session()

#start an environment
env = TicTacToe.TicTacToe()

number_updates_per_expert_update = 10
number_expert_updates = 4

#Define Loss functions *symbolically*
log_prob = get_log_prob(new_model, action_placeholder, mask_placeholder)
loss, update_op, optimizer = loss_and_update_op(log_prob, adv_n_placeholder)

#Initialize all variables first time only, otherwise update only uptimizer vars
sess.run(tf.global_variables_initializer())

for i in range(20):
        #Produce some trajectories
        paths = sample_trajectories_random_opponent(new_model, sess, 20, env = env)
        adv_n = sum_of_rewards(paths) 
        boards = np.concatenate([path['observation'] for path in paths])
        actions = np.squeeze(np.concatenate([path["action"] for path in paths])).astype(int)
        test_vars =  tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="model" +str(-1))
        for var in test_vars:
            print(var.name, np.max(sess.run(var)))
        sess.run(update_op, feed_dict = {adv_n_placeholder: adv_n, board_placeholder: boards , action_placeholder: actions})


TypeError: get_log_prob() missing 1 required positional argument: 'mask_placeholder'

In [18]:
def sample_trajectory(player, opponent, env):
    """player:   realization of Player.Player abstract class
       opponent: realization of Player.Player abstract class
       env:      environment which follows open ai gym environment structure and has a current_player int either 1 or 2
    
       realizes a rollout of env using player and opponent policy
       
       ouputs a path dictionary with keys: observation, reward, action
       Each yields a 1D np array which shows the observation, reward, action pair at every point of the rollout
    """
    
    obs, acs, rewards = [], [], []
    ob = env.reset()
    while True:
        if env.current_player == 1:
            obs.append(ob)
            player_action = player.policy(np.array([ob]), env.legal_moves)
            acs.append(player_action[0])
            ob, rew, done, _ = env.step(player_action[0])
            rewards.append(rew)
            
            if done:
                break
        else:
            opponent_action = opponent.policy(np.array([ob]), env.legal_moves())
            ob, rew, done, _ = env.step(opponent_action[0])
                
    path = {"observation" : np.array(obs, dtype=np.float32), 
                "reward" : np.array(rewards, dtype=np.float32), 
                "action" : np.array(acs, dtype=np.float32)}
    return path


In [9]:
from importlib import reload
reload(Players)
player = Players.Random_Player()
opponent = Players.Expert_Player()

env = TicTacToe.TicTacToe()
batch_winners = {0:0,1:0,2:0}
for i in range(10000):
    path = sample_trajectory(player,opponent,env)
    batch_winners[env.current_winner] += 1
print(batch_winners)
    

{0: 867, 1: 0, 2: 9133}


In [32]:
#Implementing Q-learning....
from importlib import reload 
reload(Players)

def collect_tuples(model, observation_placeholder, opponent, env, sess, num_tuples =100, exploration =.1):
    current_state1 = None
    current_action = None
    current_reward = None
    current_state2 = None
    tuples = [] #or maybe an array?
    ob = env.reset
    i=0
    while i< num_tuples:
        i+=1
        #Record the current state
        current_state1 = ob

        #We pick the best action epsilon-greedily
        action = sess.run( model, feed_dict = {observation_placeholder: [ob]})[0] #or something
        if np.random.uniform() < exploration:
            action = env.action_space.sample()
        
        #Record the current action and reward
        ob = env.step(action)
        current_action = action
        current_reward = env.get_reward() #or whatever
        
        #Record the new state
        current_state2 = ob
        tuples.append({'state1': current_state1, 'action':current_action, 'state2':current_state2, 'reward':current_reward})
        
    return tuples
    
    
def Q_learning_model(state1_placeholder, scope, num_actions = 9, reuse=tf.AUTO_REUSE):
    # A model for a TicTacToe q-function
    with tf.variable_scope(scope, reuse=reuse):
        out = tf.contrib.layers.flatten(state1_placeholder)
        out = tf.cast(out, tf.float32)
        out = tf.layers.dense(out, 64  , bias_initializer = tf.zeros_initializer(), activation = tf.nn.softmax)
        out = tf.layers.dense(out, 64  , bias_initializer = tf.zeros_initializer(), activation = tf.nn.softmax)
        Q_func = tf.layers.dense(out, num_actions , kernel_initializer = tf.zeros_initializer(), bias_initializer = tf.zeros_initializer(), activation = None)
    return Q_func


def get_loss_and_optimizer(model, targets_placeholder):
    diff = tf.subtract(tf.cast(model, tf.float64), tf.cast(targets_placeholder, tf.float64))
    loss = tf.reduce_sum( tf.square( diff ), 1  ) 
    optimizer = tf.train.AdamOptimizer(5e-3)
    update_op = optimizer.minimize(loss)
    return loss, update_op, optimizer




    

In [34]:
tf.reset_default_graph()


#Define any constants
gamma = .95

#Define the placeholders
state1_placeholder = tf.placeholder(shape = [None, 3,3], dtype = tf.int32)
action_placeholder = tf.placeholder(shape = [None], dtype = tf.int32)
state2_placeholder = tf.placeholder(shape = [None, 3,3], dtype = tf.int32)
targets_placeholder = tf.placeholder(shape = [None, 9], dtype = tf.int32)
mask_placeholder = tf.placeholder(shape = [None,9], dtype = tf.int32)


#Define the model and loss function
model = Q_learning_model(state1_placeholder, scope = "firstQ",  num_actions = 9, reuse=tf.AUTO_REUSE)
_, update_op, _  = get_loss_and_optimizer(model, targets_placeholder)

#Start a session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#Define the players
player = Players.Q_Player(model, sess, state1_placeholder, mask_placeholder, duplicate=True)
opponent = Players.Random_Player()

#Collect some tuples
database = []
database += collect_tuples(model, state1_placeholder, opponent, env, sess)

###Training time
#I have a tuple, state-action-state from somewhere
sampled_tuple = random.choice(database)

not_updated_Q = sess.run(player.q_func(), feed_dict = {state1_placeholder: [sampled_tuple['state1']]})

target_Q = not_updated_Q
target_Q[0,sampled_tuple['action']] =  sampled_tuple['reward'] 
target_Q[0,sampled_tuple['action']] += tf.reduce_max(sess.run(player.q_function(), feed_dict = {state1_placeholder: [sampled_tuple['state2']]}), 1)[0]

sess.run(update_op, feed_dict= {state1_placeholder: sampled_tuple['state1'], targets_placeholder: [target_Q] })




# def get_target_values(player, gamma = .99):
#     #sample from replay buffer
#     #get form state-action-state
#     initial_state = 
#     action = 
#     final_state =
    
#     state_action_value = player.policy(initial_state)[action]
    
    
    
    
    


duplicating session to freeze weights for evaluation...
INFO:tensorflow:Restoring parameters from ./to_duplicate.ckpt
Destroying Q_Player and Session...


TypeError: int() argument must be a string, a bytes-like object or a number, not 'method'